# Section 4. Agents

"An AI agent is a system that uses an LLM to decide the control flow of an application."
— Harrison Chase, co-founder of LangChain

In other words, an agent is a system that uses a language model to determine the control flow of an application.

In the context of LangChain, agents are components that use language models to make decisions about which actions to take and in what order. Unlike chains, where the sequence of actions is predefined, agents use the language model as a reasoning engine to dynamically decide the actions to be taken.

LangChain agents are designed to interact with various tools, process inputs, and generate meaningful outputs. Instead of providing static responses, agents follow a looped structure to perform reasoning and iterative steps until the desired output is reached.

This dynamic approach allows agents to handle complex queries and adapt to different scenarios, making them essential components for building more intelligent and adaptable AI applications.

In [ ]:
# !pip install dotenv
# !pip install langchain_openai
# !pip install -qU langchain-google-genai
# !pip install -U wikipedia
# !pip install -U numexpr
# !pip install langchain_community
# !pip install langgraph
# !pip install DateTime

In [ ]:
import getpass
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

def get_model_name(model_name, temperature=0):
    if model_name == "gemini": # https://ai.google.dev/gemini-api/docs/rate-limits?hl=pt-br
        if "GOOGLE_API_KEY" not in os.environ: # https://ai.google.dev/gemini-api/docs/api-key
            os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
        llm = ChatGoogleGenerativeAI(
            # model="gemini-1.5-pro", # max 50 / dia
            model="gemini-1.5-flash", # max 1500 / dia
            temperature=temperature,
        )
    elif model_name == "openai":
        if "OPENAI_API_KEY" not in os.environ: # https://platform.openai.com
            os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=temperature,
        )
    return llm

llm = get_model_name('gemini')

## 4.1 Tools

Tools are external functions that agents can use to perform specific tasks. They work like “skills” that expand what the agent is capable of doing. A well-defined agent is aware of which tools are available and understands how to use them—that is, what inputs and outputs are expected for the function defining the tool.

LangChain offers many built-in tools that can simplify the implementation of agent pipelines. In this lesson, we’ll look at some of them and then learn how to create a new tool.

Built-in LangChain tools: https://python.langchain.com/docs/integrations/tools/

Naturally, a Large Language Model (LLM) does not, by itself, have the knowledge needed to call functions or interact directly with the external environment. For this, it’s essential that the LLM’s output is well structured and follows a defined format that can be interpreted by the system.

To enable an LLM to act as an agent, we use a prompting technique known as ReAct (Reasoning + Acting), which we studied earlier. With this method, the LLM is able to reason through the task and, based on that, decide what action to take—such as calling a specific tool.



Now, in the cell below, we’ll begin exploring how to define tools and create a ReAct-style agent. This type of agent is capable of intelligently selecting and using tools before returning a response to us.

First, we import two important libraries:
* `load_tools`, which loads built-in LangChain tools—in this case, a calculator and a tool that accesses Wikipedia.

* `create_react_agent`, which creates a ReAct-style agent using LangGraph as the executor.

Later on, we’ll study LangGraph in more depth. But in summary, it allows you to design an execution graph through which the agent can navigate while reasoning, until it decides it has found the answer and ends execution.

In [ ]:
from langchain.agents import load_tools
from langgraph.prebuilt import create_react_agent

tools = load_tools(["llm-math","wikipedia"], llm=llm)

langgraph_agent_executor = create_react_agent(llm, tools)

Now, let’s test our agent with a math question.

In the last cell, all the actions taken by the LLM to answer the prompt are available in the `"messages"` key of the return.

By analyzing these messages, we see that the first is our original prompt sent to the LLM. Then, it makes a `function_call` with the name `Calculator`. Right after that, the `Calculator` tool responds with the result of the calculation. Finally, the LLM uses this information to formulate the final answer.

Cool — we were able to see the sequence of actions taken by the LLM. However, the format isn’t very clear, and the visualization can be a bit confusing, which makes debugging or understanding what happened more difficult.

Wouldn’t it be great if there was some kind of IDE to track all of this in a more visual way?

In [ ]:
query = "What is the 25 percent of 300?"
response = langgraph_agent_executor.invoke({"messages": [("human", query)]})

# Show response
for msg in response["messages"]:
    print(msg)

content='What is the 25 percent of 300?' additional_kwargs={} response_metadata={} id='c5ce7219-c96e-4494-a04a-44cafa69c8d5'
content='' additional_kwargs={'function_call': {'name': 'Calculator', 'arguments': '{"__arg1": "25 percent of 300"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-8a5ec94e-3c53-4fcc-bdcd-7967e61046b1-0' tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '25 percent of 300'}, 'id': '6f494fe1-f15e-465a-ab50-ffacc9ac23aa', 'type': 'tool_call'}] usage_metadata={'input_tokens': 77, 'output_tokens': 12, 'total_tokens': 89, 'input_token_details': {'cache_read': 0}}
content='Answer: 75.0' name='Calculator' id='e20c2faf-0c27-4f78-84b9-29120a7fda1f' tool_call_id='6f494fe1-f15e-465a-ab50-ffacc9ac23aa'
content='25 percent of 300 is 75.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_rea

## 4.2 LangSmith

LangSmith is a platform created by the LangChain team with the goal of helping developers debug, monitor, evaluate, and improve applications based on language models (LLMs).

When working with complex agents and chains, it can be difficult to understand exactly what’s happening behind the model’s decisions. That’s where LangSmith comes in—as a sort of “lab” for experimentation and observability for LLMs.

Among its features, the most commonly used is tracing, which allows you to view in detail how a chain or agent executed a task—from the user input to tool calls and the final generated responses.

To use LangSmith, first create an account on the official site: [LangSmith](https://smith.langchain.com/). Then, generate a new API key from your account.

Activating LangSmith is very simple: just set the environment variable `LANGSMITH_TRACING` to `true`. With that, executions from LangChain and LangGraph will automatically be synced to LangSmith, in the project specified by the `LANGSMITH_PROJECT` variable.

In [ ]:
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "insper_llm"

In [ ]:
query = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
response = langgraph_agent_executor.invoke({"messages": [("human", query)]})

# Show response
for msg in response["messages"]:
    print(msg)

content='Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?' additional_kwargs={} response_metadata={} id='d3ba48a2-5462-4c2e-906a-4fb64f3461f6'
content='I am sorry, I do not have access to external websites or specific information about authors and their works.  Therefore, I cannot tell you what book Tom M. Mitchell wrote.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-363ecab1-e0d9-49ef-bba7-97b8c9ae9d74-0' usage_metadata={'input_tokens': 88, 'output_tokens': 37, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}}


## 4.3 Python REPL Agent

The Python REPL Agent is a type of agent that has the ability to execute Python code in real time. It’s extremely useful when you want the LLM to handle tasks involving more complex logic, precise mathematical calculations, list manipulation, dates, or any other type of detailed programming.

REPL stands for:

Read–Eval–Print Loop — in other words, an environment where the code is read, executed, and the result is returned immediately.

The execution environment is controlled, but it's still important to remember: running arbitrary code generated by an LLM requires caution, especially in production environments, so always be careful when using REPL agents.

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
Thought: I need to sort the list of lists based on the last name and then the first name.  I'll use the `sorted()` function with a custom key.
Action: Python_REPL
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[0], x[1]))
print(sorted_customers)
```
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:Thought: I now know the final answer
Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]"}

From the execution, we can see that the agent first generated Python code to solve the problem. Then, that code was executed in the REPL environment, and the resulting output was returned to us, the user.

### 4.3.1 PythonREPL + create_react_agent

Now, let’s repeat the same process using create_react_agent from LangGraph.
This time, we start by defining the PythonREPL object and then associating it with a Tool:

```
# Tool
python_repl = PythonREPL()
print(python_repl.run("print(1+1)"))
tools = [Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)]
```

After that, we define the system prompt and create the agent.

It's worth noting that, due to some limitations in ReAct-style prompting, this cell may not always work correctly with Gemini. This highlights one of the advantages of the OpenAI API, which is currently more optimized to handle structured outputs and the ReAct method more consistently.


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_experimental.utilities import PythonREPL
from langchain.schema import HumanMessage
from langchain_core.tools import Tool
import langchain

llm_openai = get_model_name('openai', temperature=0.1)

# Tool
python_repl = PythonREPL()
print(python_repl.run("print(1+1)"))
tools = [Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)]

# System message
system_message = "You are a helpful assistant. Use the tools to answer user questions."

# Agent executor
langgraph_agent_executor = create_react_agent(llm_openai, tools, prompt=system_message)

# Data
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]]

# Prompt
message = f"""Use the Python tool to sort this list of customers by last name, then first name, and print the result:\n{customer_list}"""

# Run
response = langgraph_agent_executor.invoke({
    "messages": [HumanMessage(content=message)]
})

# Show response
for msg in response["messages"]:
    print(msg)

2

content="Use the Python tool to sort this list of customers by last name, then first name, and print the result:\n[['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]" additional_kwargs={} response_metadata={} id='7033a92a-8eb0-4e37-b6d1-b3bbc2e46f67'
content='' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'MALFORMED_FUNCTION_CALL', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-4457557e-cdff-47a0-b91f-f3d19f16be22-0' usage_metadata={'input_tokens': 131, 'output_tokens': 0, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}}


Besides LangSmith, another way to view an agent’s execution in detail is as shown in the cell below:

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## 4.4 Pandas Dataframe Agent

Another type of agent used in many applications is the [Pandas Dataframe Agent](https://python.langchain.com/docs/integrations/tools/pandas/). This agent specializes in generating code to analyze tables (dataframes) that are directly attached to the agent. It uses Python’s pandas library to manipulate and interpret the data, and then executes that code through the Python REPL, delivering the requested analysis to the user.

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    # agent_type=AgentType.OPENAI_FUNCTIONS, # only for OpenAI models
    verbose=True,
    allow_dangerous_code=True, # for security - will only work if True
    )
agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891There are 891 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 891 rows in the dataframe.'}

## 4.5 How To Create Tools

LangChain already provides a wide range of ready-to-use tools—external APIs, databases, browsers, etc. That’s why it’s highly recommended to explore the full list of available tools, along with usage examples, directly in the official LangChain documentation: [link](https://python.langchain.com/docs/integrations/tools/).

In many situations, we need to create custom tools to meet specific needs in our agent applications. The simplest way to do this is by using the `@tool` decorator.

By default, the decorator uses the function name as the tool name, but this can be overridden by passing a string as the first argument. In addition, it uses the function’s docstring as the tool’s description — so it’s mandatory to provide an appropriate docstring.

In the cell below, we define a simple function called `time`, which returns the current time. As mentioned, when using the `@tool` decorator, it’s important to define both the docstring and input/output type annotations, as this information is automatically passed to the agent so it knows what the tool does and how to use it.

Finally, we print the tool’s name, parameters, and description, showing exactly how they will be presented to the agent.


In [ ]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


print(time.name)
print(time.description)
print(time.args)

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant, always start a answer with 'Bro...' and end with 'got it?' "),
        ("placeholder", "{messages}"),
    ]
)

## or simply use the prompt template as a string
# system_message = "You are a helpful assistant, always start a answer with 'Bro...' and end with 'got it?' "

tools = load_tools(["llm-math","wikipedia"], llm=llm)
tools = tools + [time]
langgraph_agent_executor = create_react_agent(llm, tools, prompt=prompt)

query = "whats the date today?"
response = langgraph_agent_executor.invoke({"messages": [("human", query)]})

# Show response
for msg in response["messages"]:
    print(msg)

content='whats the date today?' additional_kwargs={} response_metadata={} id='2d2eaa0a-41b6-46a3-817c-30d0d672d00e'
content='Bro...' additional_kwargs={'function_call': {'name': 'time', 'arguments': '{"text": ""}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-62263a07-bbee-41de-af9c-9dd94f86aa25-0' tool_calls=[{'name': 'time', 'args': {'text': ''}, 'id': '7d07191b-c475-4aca-bffa-ae3939ed0a83', 'type': 'tool_call'}] usage_metadata={'input_tokens': 127, 'output_tokens': 4, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}}
content='2025-03-29' name='time' id='69c07920-ae63-474d-a842-b3106db37e6a' tool_call_id='7d07191b-c475-4aca-bffa-ae3939ed0a83'
content="Bro...Today's date is 2025-03-29. got it?" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-fla

## 4.5.1 More On Tool Creation

There are two other ways to define a custom tool—check out the LangChain tutorial to learn more:

[How to create tools | 🦜️🔗 LangChain](https://python.langchain.com/docs/how_to/custom_tools/)